In [ ]:
# Python imports
from trino.dbapi import connect
from trino.auth import JWTAuthentication
import os
import pandas as pd
from client.device_auth import get_token
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import getpass

In [ ]:
def decrypt_openssl_aes256cbc_b64(ciphertext_b64: str,
                                  password: str,
                                  iterations: int = 69) -> bytes:
    """
    Decrypts an OpenSSL-style salted, base64’d AES-256-CBC blob
    that was encrypted with:
       openssl enc -aes-256-cbc -a -salt -pbkdf2 -iter 69 -pass pass:<password>
    """
    # 1) Base64 → raw
    data = base64.b64decode(ciphertext_b64)
    
    # 2) Pull out the 8-byte OpenSSL header + 8-byte salt
    if not data.startswith(b"Salted__"):
        raise ValueError("Invalid data: missing 'Salted__' header")
    salt = data[8:16]
    encrypted = data[16:]
    
    # 3) Derive key+iv with PBKDF2-HMAC-SHA256
    #    total length = 32 bytes key + 16 bytes IV = 48
    key_iv = hashlib.pbkdf2_hmac(
        hash_name="sha256",
        password=password.encode("utf-8"),
        salt=salt,
        iterations=iterations,
        dklen=48
    )
    key, iv = key_iv[:32], key_iv[32:]
    
    # 4) AES-256-CBC decrypt + PKCS#7 unpad
    cipher = AES.new(key, AES.MODE_CBC, iv)
    padded = cipher.decrypt(encrypted)
    return unpad(padded, AES.block_size)

In [ ]:
key = getpass.getpass(prompt='DEC KEY: ')

In [ ]:
url = decrypt_openssl_aes256cbc_b64("U2FsdGVkX19Ad+HyCor1IbWVS+wRdUii1vKMSoDUs0/isLBuQlEK1GaYu6jwVqTX",key).decode('utf-8')
client_id = decrypt_openssl_aes256cbc_b64("U2FsdGVkX1/WOCYB92XZPBNAJR4bkKx7UzSwlMyfBBde4j2c8trc8nVkCiaQCDMGLMS7RVrN8HZ9KHypDRWxGg==",key).decode('utf-8')
secret_id = decrypt_openssl_aes256cbc_b64("U2FsdGVkX1/njd4MUV76e6FwSG7uBckeVh509i9ujV1wGLYQ3I3YIh2ivbuOJR7wT0Q8gdOYmvcEP/nGmRT7jzEKvUPBHYaT1RrQWgiMGj3kBQonoMQjb8yC0hKdIyAN",key).decode('utf-8')
device_url = decrypt_openssl_aes256cbc_b64("U2FsdGVkX18JFe4wsQCi3Fj343EwYHpT7vpXXbpPHfBN4pERQ/Ygk1N2IDBf0pmuqUreCMkqQAwcbNqqE43BwlLh5aND5RE0lqRjfYqE4ns=",key).decode('utf-8')
token_url = decrypt_openssl_aes256cbc_b64("U2FsdGVkX1/mI/fSgyUkYfO4DegvF/eKc0lT+kT4DcfKhzuJtXue5N6ZgQb1ZHYm9+CSLul9WNStgqFT1cetPdS7D0uusNioEpA3XI9ugrw=",key).decode('utf-8')

In [ ]:
bsso_token = get_token("uat", client_id, secret_id, device_url, token_url)

In [ ]:
# Create a connection to Trino
conn = connect(
    auth=JWTAuthentication(bsso_token),
    http_scheme="https",
    host=url,
    port=443
)

In [ ]:
sql='show catalogs'

In [ ]:
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall()

df = pd.DataFrame(
    data=rows, columns=[detail[0] for detail in cur.description]
)

In [ ]:
df